## Способы борьбы с несбалансированными классами
Eсть два довольно важных момента, которые нужно
повторить, потому что некоторые специалисты (в особенности начинающие), как правило,
игнорируют их. Первый момент связан с перекрестной проверки.
Перекрестная проверка или использование тестового набора позволяют
оценить модель машинного обучения с точки зрения того, как она будет
работать в будущем. Однако, если мы с помощью тестового набора или
перекрестной проверки осуществляем отбор модели или отбор
параметров модели, мы «растрачиваем» тестовые данные, а
использование тех же самых данных для оценки работы модели в
будущем приведет к чрезмерно оптимистичным прогнозам. Поэтому нам
необходимо разбить данные на обучающий набор для построения модели,
проверочный набор для отбора модели параметров и тестовый набор для
оценки качества моделей. Вместо одного разбиения мы можем
использовать разбиения перекрестной проверки. Наиболее часто
используемым вариантом (как описывалось ранее) является разбиение
обучение/тест для оценки, а также использование перекрестной
проверки на обучающем наборе для отбора модели и параметров.
Второй момент связан с важностью метрики качества или функции
оценки, которые используются для отбора модели и оценки модели.
Однако в проектах машинного обучения построение
модели с высоким значением правильности редко бывает конечной
целью. Надо убедиться в том, что метрика, используемая для оценки и отбора
модели, является точным приближением решаемой задачи. В реальности
классификационные задачи редко характеризуются
сбалансированностью классов и зачастую ложно положительные и ложно
отрицательные примеры ведут к совершенно различным последствиям.

## Методы борьбы с несбаллансированными классами
### 1)AUC-ROC 
### 2)AUC-PR
### 3)Logistic Loss
### 4)KNeighborsClassifier

Примеры несбланасированных классов:
медицинская диагностика, обнаружение мошеннических транзакция, классификация текстов

Основная проблема, связанная с несбалансированными выборками, состоит в том, что классификаторы минимизируют число неправильных ответов и никак не учитывают цены ошибок. Может возникнуть ситуация,
когда выгоднее отнести все объекты к большему классу, не пытаясь как-то выделить объекты маленького
класса. Другими словами, при работе с несбалансированными выборками классификаторы получаются очень
плохие с точки зрения точности или полноты.

## Метрики качества в задачах регрессии
### Среднеквадратичная ошибка
### Средняя абсолютная ошибка
### Коэффициент детерминации
### Несимметричные потери

## Метрика качества классификации
### Доля правильных ответов
### Несбалансированные выборки
### Цены ошибок
#### Выбор метрик влияет на то, как измеряется и сравнивается производительность алгоритмов машинного обучения. Они влияют на то, как вы оцениваете важность различных характеристик в результатах и   ваш окончательный выбор того, какой алгоритм выбрать.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve
data = pd.read_csv('creditcard.csv')

data.head()

In [ ]:
data.isnull().sum().max() # проверка на null

In [ ]:
print(data['Class'].value_counts())

In [ ]:
data.describe()

### Судя по данным они очень несбалансированные и выдают большинство операций за не мошеннические

### В данном случае нужно масштабировать Time и Amount

In [ ]:
from sklearn.preprocessing import RobustScaler


rob_scaler = RobustScaler()

data['scaled_amount'] = rob_scaler.fit_transform(data['Amount'].values.reshape(-1,1))
data['scaled_time'] = rob_scaler.fit_transform(data['Time'].values.reshape(-1,1))
data.drop(['Time','Amount'], axis=1, inplace=True)

In [ ]:
data.head()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(18,4))

amount_val = data['scaled_amount'].values
time_val = data['scaled_time'].values

sns.distplot(amount_val, ax=ax[0], color='y')
ax[0].set_title('scaled_amount', fontsize=14)
ax[0].set_xlim([min(amount_val), max(amount_val)])

sns.distplot(time_val, ax=ax[1], color='g')
ax[1].set_title('scaled_time', fontsize=14)
ax[1].set_xlim([min(time_val), max(time_val)])

plt.show()

### Данные я разделю методом cross-validation

In [ ]:
data = data.sample(frac=1)

fraud_data = data.loc[data['Class'] == 1]
non_fraud_data = data.loc[data['Class'] == 0][:492]

normal_distributed_data = pd.concat([fraud_data, non_fraud_data])
new_data = normal_distributed_data.sample(frac=1, random_state=42)

new_data.head()

In [ ]:
X = new_data.drop('Class', axis=1)
y = new_data['Class']

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_predict
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values


### В данном случае я буду использовать логическую регрессию и метод К ближайших соседей

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

logReg = LogisticRegression()
logReg.fit(X_train, y_train)
cross_val_score(logReg, X_train, y_train, cv=5).mean()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

KN = KNeighborsClassifier()
KN.fit(X_train, y_train)
cross_val_score(KN, X_train, y_train, cv=5).mean()

In [ ]:
def logistic_roc_curve(log_fpr, log_tpr):
    plt.figure(figsize=(12,8))
    plt.title('Logistic Regression ROC', fontsize=16)
    plt.plot(log_fpr, log_tpr, 'b-', linewidth=2)
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)
    plt.axis([-0.01,1,0,1])
    
log_reg_pred = cross_val_predict(logReg, X_train, y_train, cv=5,
                             method="decision_function")
log_fpr, log_tpr, log_thresold = roc_curve(y_train, log_reg_pred)
logistic_roc_curve(log_fpr, log_tpr)
plt.show()

### При перекрестной проверке значения оказались высоки и у LogisticRegression и у KNeighborsClassifier и у других классификаторов они должны быть высоки

In [ ]:
from sklearn.metrics import classification_report
y_pred_log_reg = logReg.predict(X_test)

y_pred_knear = KN.predict(X_test)

print('Logistic Regression:')
print(classification_report(y_test, y_pred_log_reg))

print('KNears Neighbors:')
print(classification_report(y_test, y_pred_knear))

### Перекрестная выборка подошла к решения проблемы с несбланасированными данными

